In [35]:
import pandas as pd
import numpy as np
import holidays
from datetime import date


In [4]:
# load city date dimensions
den_date = pd.read_csv('den_dateDim.csv')
van_date = pd.read_csv('van_dateDim.csv')

In [56]:
# drop duplicates 
van_date = van_date.drop_duplicates().sort_values('timestamp').reset_index()
van_date

,level_0,index,dayOfWeek,dayOfMonth,month,year,isWeekend,timestamp
0,143,189,3,1,1,2003,0,2003-01-01
1,82,98,4,2,1,2003,0,2003-01-02
2,74,82,5,3,1,2003,0,2003-01-03
3,303,676,6,4,1,2003,1,2003-01-04
4,114,144,7,5,1,2003,1,2003-01-05
...,...,...,...,...,...,...,...,...
5303,5168,512857,7,9,7,2017,1,2017-07-09
5304,5303,513540,1,10,7,2017,0,2017-07-10
5305,5130,512815,2,11,7,2017,0,2017-07-11
5306,5153,512839,3,12,7,2017,0,2017-07-12


In [57]:
# drop duplicates
den_date = den_date.drop_duplicates().sort_values('timestamp').reset_index()
den_date

,level_0,index,dayOfWeek,dayOfMonth,month,year,isWeekend,timestamp
0,1050,66971,5,2,1,2015,0,2015-01-02
1,1083,71789,6,3,1,2015,1,2015-01-03
2,1251,86168,7,4,1,2015,1,2015-01-04
3,1352,95516,1,5,1,2015,0,2015-01-05
4,1023,66243,2,6,1,2015,0,2015-01-06
...,...,...,...,...,...,...,...,...
1858,973,61282,1,3,2,2020,0,2020-02-03
1859,971,61223,2,4,2,2020,0,2020-02-04
1860,969,61193,3,5,2,2020,0,2020-02-05
1861,967,61183,4,6,2,2020,0,2020-02-06


In [105]:
den_date['timestamp'][924]

'2017-07-14'

In [106]:
# extract columns
dayOfWeek_den = den_date['dayOfWeek']
dayOfMonth_den = den_date['dayOfMonth']
month_den = den_date["month"]
year_den = den_date['year']
isWeekend_den = den_date['isWeekend']
timestamp_den = den_date['timestamp']

dayOfWeek_van = van_date['dayOfWeek']
dayOfMonth_van = van_date['dayOfMonth']
month_van = van_date["month"]
year_van = van_date['year']
isWeekend_van = van_date['isWeekend']
timestamp_van = van_date['timestamp']

In [107]:
# organizing data for concating
dfden_date = pd.DataFrame({'dayOfWeek':dayOfWeek_den, 'dayOfMonth':dayOfMonth_den, 
                          'month':month_den, 'year':year_den,
                         'isWeekend':isWeekend_den,'timestamp':timestamp_den})

dfvan_date = pd.DataFrame({'dayOfWeek':dayOfWeek_van, 'dayOfMonth':dayOfMonth_van, 
                          'month':month_van, 'year':year_van,
                         'isWeekend':isWeekend_van,'timestamp':timestamp_van})


#concatanating city date dimensions
df_date = pd.concat([dfvan_date, dfden_date[924:1864]])
df_date = df_date.drop_duplicates().sort_values('timestamp').reset_index()
df_date = df_date.drop(columns = ['index'])

df_date

,dayOfWeek,dayOfMonth,month,year,isWeekend,timestamp
0,3,1,1,2003,0,2003-01-01
1,4,2,1,2003,0,2003-01-02
2,5,3,1,2003,0,2003-01-03
3,6,4,1,2003,1,2003-01-04
4,7,5,1,2003,1,2003-01-05
...,...,...,...,...,...,...
6242,1,3,2,2020,0,2020-02-03
6243,2,4,2,2020,0,2020-02-04
6244,3,5,2,2020,0,2020-02-05
6245,4,6,2,2020,0,2020-02-06


In [108]:
# extracting new columns
dayOfWeek = df_date['dayOfWeek']
dayOfMonth = df_date['dayOfMonth']
month = df_date["month"]
year = df_date['year']
isWeekend = df_date['isWeekend']
timestamp = df_date['timestamp']

df_count = len(df_date)

isHoliday_US = np.full(df_count,'0')
isHoliday_CA = np.full(df_count,'0')

# labelling dates as US or CA holidays
for i in range(len(isHoliday_den)):
    dat = timestamp[i]
    holCA = holidays.CA().get(dat)
    holUS = holidays.US().get(dat)
    if holCA != None:
        isHoliday_CA[i] = 1
    else:
        isHoliday_CA[i] = 0
    if holUS != None:
        isHoliday_US[i] = 1
    else:
        isHoliday_US[i] = 0

# creating final date dimension
df_date = pd.DataFrame({'dayOfWeek':dayOfWeek, 'dayOfMonth':dayOfMonth, 
                          'month':month, 'year':year,
                         'isWeekend':isWeekend,'timestamp':timestamp,
                          'isHolidayCA':isHoliday_CA, 'isHolidayUS':isHoliday_US})

In [109]:
df_date

,dayOfWeek,dayOfMonth,month,year,isWeekend,timestamp,isHolidayCA,isHolidayUS
0,3,1,1,2003,0,2003-01-01,1,1
1,4,2,1,2003,0,2003-01-02,0,0
2,5,3,1,2003,0,2003-01-03,0,0
3,6,4,1,2003,1,2003-01-04,0,0
4,7,5,1,2003,1,2003-01-05,0,0
...,...,...,...,...,...,...,...,...
6242,1,3,2,2020,0,2020-02-03,0,0
6243,2,4,2,2020,0,2020-02-04,0,0
6244,3,5,2,2020,0,2020-02-05,0,0
6245,4,6,2,2020,0,2020-02-06,0,0


In [110]:
df_date.to_csv('date_dimension.csv')